In [1]:
import numpy as np
import pandas as pd
import numpy as np
import os
import time
from google.cloud import bigquery
import pandas_gbq
import glob

In [2]:
client = bigquery.Client()

In [5]:
query = "select * from `repositoriodedadosgpsp.acoes_afirmativas.SIAPE_perc_professor_superior`ORDER BY universidade"  

In [6]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Downloading: 100%|██████████|


In [ ]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar")

In [6]:
df.to_csv("SIAPE_perc_professor_superior.csv",index=False, encoding="latin-1", sep=';', decimal=',')

-----------------------------------------------------------

In [5]:
df1= pd.read_csv("RAIS_remuneracao_vinculos_publicos_privados2.csv")

In [49]:
df.head(15)

,ano,quantidade_vinculos,prop_ano,esfera,poderes
0,1985,849283,0.175573,Federal,Executivo
1,1985,2319284,0.479469,Estadual,Executivo
2,1985,1546665,0.319744,Municipal,Executivo
3,1985,14330,0.002962,Federal,Legislativo
4,1985,19595,0.004051,Estadual,Legislativo
5,1985,15051,0.003112,Municipal,Legislativo
6,1985,23816,0.004924,Federal,Judiciário
7,1985,49170,0.010165,Estadual,Judiciário
8,1986,886308,0.173698,Federal,Executivo
9,1986,2389021,0.468199,Estadual,Executivo


In [50]:
df.to_csv("RAIS_poder_nivel_vinculos.csv",index=False, encoding="latin-1", sep=';', decimal=',')

In [34]:
df1= pd.read_csv("RAIS_remuneracao_vinculos_publicos_privados2.csv",encoding="latin-1", sep=';', decimal=',')

Automatizando troca de csv

In [7]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar")

In [ ]:
## baixando os atuais com , e exportando com ";"

In [15]:
## listando todos os itens do diretório
len(glob.glob('*.csv'))

42

In [17]:
## Serve apenas na primeira vez . 

for i in list(range(0,len(glob.glob('*.csv')))):
    
    df= pd.read_csv(glob.glob('*.csv')[i])
    df.to_csv(glob.glob('*.csv')[i],index=False, encoding="latin-1", sep=';', decimal=',')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 139: invalid continuation byte